In [55]:
import toy_data.cov_shift as data_gen
import toy_data as td
import bokeh.io
import numpy as np
import bokeh.plotting as bp
import tensorflow as tf
bokeh.io.output_notebook()

Loading BokehJS ...

In [69]:
# m_ = td.models.LinearBinary(2)
m_ = td.models.rotatedSine2D(phase=np.pi/2, frequency=1)
data = data_gen.Gaussian_Shift_2D_BinaryClassification(m_, tst_X_mean_shift=(-1.7, -0.8), noise_sd=0.2)
data_gen.visualize_2D_classification(data, classifyF=m_, fig_width=800)

### Using logistic regression for classification

In [70]:
tr = data.tr
tst = data.tst

x = tf.placeholder(tf.float32, [None, 2])
W = tf.Variable(tf.random_uniform(shape=[2, 1]), dtype=tf.float32)
b = tf.Variable(tf.random_uniform(shape=[1]), dtype=tf.float32)
logits = tf.matmul(x, W) + b
y = tf.nn.sigmoid(logits)
y_ = tf.placeholder(tf.float32, [None, 1])

loss1 = tf.nn.sigmoid_cross_entropy_with_logits(logits, y_)
cross_entropy = tf.reduce_mean(loss1)
# train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)
train_step = tf.train.RMSPropOptimizer(0.01).minimize(cross_entropy)

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

for i in range(1000):
    sess.run(train_step, feed_dict={x: tr.X, y_: tr.y})
    
correct_prediction = tf.equal(y > 0.5, y_>0.5)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(sess.run(cross_entropy, feed_dict={x: tr.X, y_: tr.y}))
print(sess.run(accuracy, feed_dict={x: tst.X, y_: tst.y}))

def classifierF(X):
    return sess.run(y, feed_dict={x: X})

data_gen.visualize_2D_classification(data, classifyF=classifierF, fig_width=800)

0.18362
0.8


## Using kernel mean matching 

## 1. Finding reweightging ratio $\beta$

### Finding kernel $k(x_i^{tr}, x_j^{tr})$  and $\kappa_i = \frac{n_{tr}}{n_{te}}\sum_{j=1}^{n_{te}} k(x_i^{tr}, x_j^{te})$ using RBF

In [71]:
def rbf_dot(A, B, sigma=1):
    """
        A, B: sample × feature
    """
    A2 = np.sum(A**2, 1).reshape(1, -1).T
    B2 = np.sum(B**2, 1).reshape(1, -1)
    distance = A2 + B2 - 2*A.dot(B.T)
    return np.exp(-distance/(2*sigma**2))

In [72]:
n_tr = tr.X.shape[0]
n_tst = tst.X.shape[0]
# tr_X = add_offset(tr.X)
# tst_X = add_offset(tst.X)
K = rbf_dot(tr.X, tr.X)
K = (K + K.T)/2
kappa = np.sum(rbf_dot(tr.X, tst.X), axis=1)*n_tr/n_tst

def heat_map(data, height=None, width=None):
    n_y, n_x = data.shape
    print(data.shape)
    p = bp.figure(
        x_range=(0, n_x),
        y_range=(0, n_y),
    )
    if height:
        p.height=height
    if width:
        p.width=width
    p.image(image=[data], dw=[n_x], dh=[n_y], palette="Spectral11")
    bp.show(p)

def plot(X):
    p = bp.figure()
    x = np.ravel(X)
    p.scatter(range(x.size), x)
    bp.show(p)
    
heat_map(K)

(160, 160)


### Formulate the optimization problem

$$
\begin{align*}
    \newcommand{\argmin}{\mathop{\mathrm{argmin}}}
    &\argmin_{\beta} &\frac{1}{n_{tr}^2} \beta^\top K \beta
    - \frac{2}{n_{tr}^2} \kappa^\top \beta \\
    &\text{s.t.} &\beta_i \in [0, B] \\
    & &\lvert \sum_{i=1}^{n_{tr}} \beta_i - n_{tr} \rvert \leq \epsilon
\end{align*}
$$

In [73]:
# B = 500
B = 10
epsilon = B/np.sqrt(n_tr)
# epsilon = np.sqrt(n_tr) - 1/np.sqrt(n_tr)

import cvxopt as opt
    
# X=solvers.qp(P, q, G, h, A, b) attempts to solve the quadratic programming problem:
#              min 0.5*x'*P*x + q'*x   
# subject to:  G*x <= h 
#              A*x = b

_P = opt.matrix(K)
_q = -opt.matrix(kappa*2)
constraint_coefs = (
    -np.eye(n_tr),
    np.eye(n_tr),
    -np.ones(n_tr),
    np.ones(n_tr)
)
_G = opt.matrix(np.vstack(constraint_coefs))
constraints = (
    np.zeros(n_tr),
    np.ones(n_tr)*B,
    np.array(n_tr*(1 - epsilon)).reshape(1),
    np.array(n_tr*(1 + epsilon)).reshape(1)
)
_h = opt.matrix(np.concatenate(constraints))
solution = opt.solvers.qp(_P, _q, _G, _h)
beta = np.array(solution['x'])
plot(beta)
data_gen.visualize_2D_classification_with_tr_weights(data, beta)

     pcost       dcost       gap    pres   dres
 0: -1.6724e+04 -2.7854e+05  8e+05  6e-01  3e-15
 1: -1.4892e+04 -2.1487e+05  4e+05  3e-01  2e-15
 2: -1.1497e+04 -7.9054e+04  7e+04  2e-16  4e-15
 3: -1.4408e+04 -2.8227e+04  1e+04  1e-15  2e-15
 4: -1.5982e+04 -2.0891e+04  5e+03  2e-16  8e-16
 5: -1.6529e+04 -1.8929e+04  2e+03  4e-16  9e-16
 6: -1.6842e+04 -1.7702e+04  9e+02  5e-16  1e-15
 7: -1.7000e+04 -1.7246e+04  2e+02  6e-16  6e-16
 8: -1.7055e+04 -1.7121e+04  7e+01  4e-16  1e-15
 9: -1.7075e+04 -1.7088e+04  1e+01  1e-15  1e-15
10: -1.7079e+04 -1.7082e+04  3e+00  1e-15  3e-15
11: -1.7080e+04 -1.7080e+04  9e-02  1e-15  2e-15
12: -1.7080e+04 -1.7080e+04  1e-03  5e-16  8e-15
Optimal solution found.


## 2. Run logistic regression with weights

$$
\begin{align*}
    &\text{minimize}_{\theta} \sum_{i=1}^{n_{tr}} - \beta_i \log p(y_i^{tr}|x_i^{tr}, \theta)\\
    =&\text{minimize}_{\theta} \sum_{i=1}^{n_{tr}} - \beta_i \log I(y_i = \hat{y_i})\\
    =&\text{minimize}_{\theta} \sum_{i=1}^{n_{tr}} - \beta_i (\log \frac{1}{1+e^{-\theta X_i}} I(y_i = 1)
    + \log (1 - \frac{1}{1+e^{-\theta X_i}}) I(y_i = 0))\\ 
\end{align*}
$$

In [74]:
x = tf.placeholder(tf.float32, [None, 2])
W = tf.Variable(tf.random_uniform(shape=[2, 1]), dtype=tf.float32)
b = tf.Variable(tf.random_uniform(shape=[1]), dtype=tf.float32)
logits = tf.matmul(x, W) + b
y = tf.nn.sigmoid(logits)
y_ = tf.placeholder(tf.float32, [None, 1])
sample_weights = tf.placeholder(tf.float32, [None, 1])

loss1 = tf.nn.sigmoid_cross_entropy_with_logits(logits, y_)
cross_entropy = tf.reduce_mean(sample_weights*loss1)
# train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)
train_step = tf.train.RMSPropOptimizer(0.01).minimize(cross_entropy)

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

for i in range(1000):
    sess.run(train_step, feed_dict={x: tr.X, y_: tr.y, sample_weights: beta})
    
correct_prediction = tf.equal(y > 0.5, y_>0.5)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(sess.run(cross_entropy, feed_dict={x: tr.X, y_: tr.y, sample_weights: beta}))
print(sess.run(accuracy, feed_dict={x: tst.X, y_: tst.y, sample_weights: beta}))

def classifierF(X):
    return sess.run(y, feed_dict={x: X, sample_weights: beta})

data_gen.visualize_2D_classification_with_tr_weights(data, beta, classifyF=classifierF, fig_width=800)

0.0435627
0.925


### RBA

#### Density estimation using logistic regression
\begin{align}
    \frac{P(X_{\text{tr}})}{P(X_{\text{tst}})} = \frac{P(X\mid s=\text{tr})}{P(X\mid s=\text{tst})}
    &= \frac{\frac{P(s=\text{tr}\mid X)P(X)}{P(s=\text{tr})}}{\frac{P(s=\text{tst}\mid X)P(X)}{P(s=\text{tst})}} \\
    &= \frac{P(s=\text{tr}\mid X)P(s=\text{tst})}{P(s=\text{tst}\mid X)P(s=\text{tr})} \\
    &= \frac{P(s=\text{tst})}{P(s=\text{tr})}\frac{1-P(s=\text{tst}\mid X)}{P(s=\text{tst}\mid X)} \\
    &= \frac{P(s=\text{tst})}{P(s=\text{tr})}(\frac{1}{P(s=\text{tst}\mid X)} - 1)
\end{align}

Hence we can find $P(s=\text{tst}\mid X)$ first and then calculate the ratio from it
\begin{align}
\text{minimize}_{w} -\log p(s=\text{tst}\mid x; w),
\end{align}
which is logistic regression on $x, s$

We just reuse the previous LR model here:

In [75]:
n_tr = tr.y.shape[0]
n_tst = tst.y.shape[0]

x = tf.placeholder(tf.float32, [None, 2])
W = tf.Variable(tf.random_uniform(shape=[2, 1]), dtype=tf.float32)
b = tf.Variable(tf.random_uniform(shape=[1]), dtype=tf.float32)
logits = tf.matmul(x, W) + b
y = tf.nn.sigmoid(logits)
y_ = tf.placeholder(tf.float32, [None, 1])

loss1 = tf.nn.weighted_cross_entropy_with_logits(logits, y_, n_tr/n_tst)
cross_entropy = tf.reduce_mean(loss1)
# train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)
train_step = tf.train.RMSPropOptimizer(0.01).minimize(cross_entropy)

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

s = np.concatenate((np.zeros(n_tr), np.ones(n_tst))).reshape(-1, 1)

for i in range(1000):
    sess.run(train_step, feed_dict={x: np.vstack((tr.X, tst.X)), y_: s})
    
correct_prediction = tf.equal(y > 0.5, y_>0.5)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(sess.run(cross_entropy, feed_dict={x: np.vstack((tr.X, tst.X)), y_: s}))
print(sess.run(accuracy, feed_dict={x: np.vstack((tr.X, tst.X)), y_: s}))

def logi_classify_F(X):
    return sess.run(y, feed_dict={x: X})

data_gen.visualize_2D_classification(data, classifyF=logi_classify_F, fig_width=400)

0.817714
0.775


* The the ratio is: 
\begin{align}
    &\frac{P(s=\text{tst})}{P(s=\text{tr})}(\frac{1}{P(s=\text{tst}\mid X)} - 1) \\
    &=\frac{P(s=\text{tst})}{P(s=\text{tr})}(\frac{1}{\text{sigmoid}(-wX)} - 1) \\
    &=\frac{P(s=\text{tst})}{P(s=\text{tr})}e^{-wX}
\end{align}

In [76]:
def logi_reweight_F(X):
    _R = sess.run(n_tst/n_tr*(tf.exp(-logits)), feed_dict={x: X})
    return _R
z = np.array([[-2,2]]).T.T
logi_reweight_F(z)

array([[ 0.1634227]], dtype=float32)

In [77]:
p = bp.figure()
p.circle(range(0, n_tr), np.ravel(logi_reweight_F(tr.X)))
bp.show(p)

In [166]:
min_eps = 0.0001
max_iter = 5000
lr = 0.01
l_theta = 0.01
theta = np.zeros((6, 1))

In [175]:
sigmoid = lambda x: .5 * (1 - np.tanh(-.5 * x))

def moments_2nd(x, y_):
    moments = (np.ones(x.shape[0]),
               x[:, 0]**2,
               x[:, 1]**2,
               x[:, 0] * x[:, 1],
               x[:, 0],
               x[:, 1]
              )
    moments = tuple(y_*m for m in moments)
    return np.vstack(moments)

m_t = moments_2nd(tr.X, True)
m_f = moments_2nd(tr.X, False)
m_c = m_t.copy()
m_c[:, np.ravel(tr.y) == False] = m_f[:, np.ravel(tr.y) == False]
beta_log = 1/logi_reweight_F(tr.X)

_iter = 0
while True:
    _iter += 1
    p_t = 1 - sigmoid(-beta_log*(m_t - m_f).T.dot(theta))
    dL = m_c.mean(axis=1, keepdims=True) - (m_t.dot(p_t) + m_f.dot(1 - p_t))/n_tr
    theta += lr * (dL - l_theta*theta)
    eps = np.linalg.norm(dL - l_theta*theta)
    if not _iter % 100:
        print(eps, np.linalg.norm(theta))
        print(theta.T)
    if eps < min_eps or _iter >= max_iter: 
        break
        
def RBA_F(x):
    _m_t = moments_2nd(x, True)
    _m_f = moments_2nd(x, False)
    # logi_reweight_F
    return 1 - sigmoid(-1/logi_reweight_F(x)*(_m_t - _m_f).T.dot(theta))

In [176]:
p = bp.figure()
p.circle(range(0, n_tr), np.ravel(RBA_F(tr.X)))
bp.show(p)

In [177]:
data_gen.visualize_2D_classification(data, classifyF=RBA_F, fig_width=800)